# Hard Problem Session

## 975. Odd Even Jump
You are given an integer array A.  From some starting index, you can make a series of jumps.  The (1st, 3rd, 5th, ...) jumps in the series are called odd numbered jumps, and the (2nd, 4th, 6th, ...) jumps in the series are called even numbered jumps.

You may from index i jump forward to index j (with i < j) in the following way:

During odd numbered jumps (ie. jumps 1, 3, 5, ...), you jump to the index j such that A[i] <= A[j] and A[j] is the smallest possible value.  If there are multiple such indexes j, you can only jump to the smallest such index j.
During even numbered jumps (ie. jumps 2, 4, 6, ...), you jump to the index j such that A[i] >= A[j] and A[j] is the largest possible value.  If there are multiple such indexes j, you can only jump to the smallest such index j.
(It may be the case that for some index i, there are no legal jumps.)
A starting index is good if, starting from that index, you can reach the end of the array (index A.length - 1) by jumping some number of times (possibly 0 or more than once.)

Return the number of good starting indexes.

 

Example 1:

Input: [10,13,12,14,15]
Output: 2
Explanation: 
From starting index i = 0, we can jump to i = 2 (since A[2] is the smallest among A[1], A[2], A[3], A[4] that is greater or equal to A[0]), then we can't jump any more.
From starting index i = 1 and i = 2, we can jump to i = 3, then we can't jump any more.
From starting index i = 3, we can jump to i = 4, so we've reached the end.
From starting index i = 4, we've reached the end already.
In total, there are 2 different starting indexes (i = 3, i = 4) where we can reach the end with some number of jumps.
Example 2:

Input: [2,3,1,1,4]
Output: 3
Explanation: 
From starting index i = 0, we make jumps to i = 1, i = 2, i = 3:

During our 1st jump (odd numbered), we first jump to i = 1 because A[1] is the smallest value in (A[1], A[2], A[3], A[4]) that is greater than or equal to A[0].

During our 2nd jump (even numbered), we jump from i = 1 to i = 2 because A[2] is the largest value in (A[2], A[3], A[4]) that is less than or equal to A[1].  A[3] is also the largest value, but 2 is a smaller index, so we can only jump to i = 2 and not i = 3.

During our 3rd jump (odd numbered), we jump from i = 2 to i = 3 because A[3] is the smallest value in (A[3], A[4]) that is greater than or equal to A[2].

We can't jump from i = 3 to i = 4, so the starting index i = 0 is not good.

In a similar manner, we can deduce that:
From starting index i = 1, we jump to i = 4, so we reach the end.
From starting index i = 2, we jump to i = 3, and then we can't jump anymore.
From starting index i = 3, we jump to i = 4, so we reach the end.
From starting index i = 4, we are already at the end.
In total, there are 3 different starting indexes (i = 1, i = 3, i = 4) where we can reach the end with some number of jumps.
Example 3:

Input: [5,1,3,4,2]
Output: 3
Explanation: 
We can reach the end from starting indexes 1, 2, and 4.

### Note:
1. 1 <= A.length <= 20000
2. 0 <= A[i] < 100000

In [272]:
# Brute Force Submmision: doesnt pass
def oddEvenJumps(A) -> int:
    # tuple that is sorted BCT: Nlogn USE SORTED would do betta
    stores = [(elem,index) for index, elem in enumerate(A)]
    cnt = 1
    for i in range(len(A)-1):
        # iterate through rest
        finished = False
        time = 1
        temp = i
        while not finished:
            if time % 2 == 0: # A[j] <= A[i], A[j] largest 
                minimum = findMin(A,A[temp],temp+1,len(A))
                if minimum == len(A) - 1:
                    finished = True
                    break
                if minimum == -1:
                    break
                temp = minimum
                time += 1
            else: # A[j] >= A[i], A[j] smallest 
                maximum = findMax(A,A[temp],temp+1,len(A))
                if maximum == len(A) - 1:
                    finished = True
                    print(A[i],A[maximum])
                    break
                if maximum == -1:
                    break
                temp = maximum
                time += 1
        if finished:
            cnt += 1
    return cnt
def findMin(arr,tar,start,end):
    res = -1
    for i in range(start,end):
        if arr[i] <= tar:
            if (res == -1) or (res != -1 and arr[i] > arr[res]):
                res = i
    return res
def findMax(arr,tar,start,end):
    res = -1
    for i in range(start,end):
        if arr[i] >= tar:
            if (res == -1) or (res != -1 and arr[i] < arr[res]):
                res = i
    return res

In [250]:
oddEvenJumps([1,1,1,1,1])

2

## Tips:
1. Common Trick: monotonic stack
```
Let's store the indices i of the processed values v0 = A[i] in a stack, and maintain the invariant that this is monotone decreasing. When we add a new index j, we pop all the smaller indices i < j from the stack, which all jump to j
```

    Code:
```
    def get_answer(A):
        answers = [None] * len(A)
        mono_stack = []
        for index in A:
            while mono_stack and mono_stack[-1] < index:
                answers[mono_stack.pop()] = index
            mono_stack.append(index)
        return answers
```
2. Backward DP
```
    for i in range(answer_size-2,-1,-1):
        if even_successor[i] is not None:
            even[i] = odd[even_successor[i]]
        if odd_successor[i] is not None:
            odd[i] = even[odd_successor[i]]
```
3. Sort index by value without adding value
```
sorted(range(answer_size),key=lambda x: A[x])
```

In [287]:
# oddEvenJumps with trick Nlogn
def oddEvenJumps(A) -> int:
    def get_answer(A):
        answers = [None] * len(A)
        mono_stack = []
        for index in A:
            while mono_stack and mono_stack[-1] < index:
                answers[mono_stack.pop()] = index
            mono_stack.append(index)
        return answers
    answer_size = len(A)
    list_indexes = sorted(range(answer_size),key=lambda x: A[x])
    odd_successor = get_answer(list_indexes)
    list_indexes.sort(key=lambda x: -A[x])
    even_successor = get_answer(list_indexes)
    # Now use Dynamic Programming to solve by going backward.
    even = [0]*answer_size
    odd = [0]*answer_size
    even[-1] = odd[-1] = 1
    for i in range(answer_size-2,-1,-1):
        if even_successor[i] is not None:
            even[i] = odd[even_successor[i]]
        if odd_successor[i] is not None:
            odd[i] = even[odd_successor[i]]
    print(odd,even)
    return sum(odd)

In [288]:
oddEvenJumps([7,6])

[0, 1] [1, 1]


1

## Article Screenshot

<img style="float: left;" src="sol1.png">
<img style="float: left;" src="sol2.png">
<img style="float: left;" src="sol3.png">

# Medium Problem Session

## 904. Fruit Into Baskets
In a row of trees, the i-th tree produces fruit with type tree[i].

You start at any tree of your choice, then repeatedly perform the following steps:

Add one piece of fruit from this tree to your baskets.  If you cannot, stop.
Move to the next tree to the right of the current tree.  If there is no tree to the right, stop.
Note that you do not have any choice after the initial choice of starting tree: you must perform step 1, then step 2, then back to step 1, then step 2, and so on until you stop.

You have two baskets, and each basket can carry any quantity of fruit, but you want each basket to only carry one type of fruit each.

What is the total amount of fruit you can collect with this procedure?

In [295]:
# Accepted Solution Beats 66.48%
class Solution:
    def totalFruit(self, tree) -> int:
        from collections import deque
        curr_max = global_max = 0
        capacity = 2
        count = dict()
        visited = deque([])
        for i in range(len(tree)):
            if tree[i] not in visited:
                if capacity > 0:
                    capacity -= 1
                else:
                    visited.popleft()
                    curr_max = count[visited[-1]]
                count[tree[i]] = 1
                visited.append(tree[i])
            else:
                if visited[-1] != tree[i]:
                    count[visited.popleft()] = 0
                    visited.append(tree[i])
                count[tree[i]] += 1
            curr_max += 1
            global_max = max(curr_max,global_max)
        return global_max

In [312]:
class Solution:
    def totalFruit(self, tree) -> int:
        from collections import Counter
        left, counter, res = 0, Counter(), 0
        for right in range(len(tree)):
            counter[tree[right]] += 1
            while len(counter) > 2:
                counter[tree[left]] -= 1
                if not counter[tree[left]]:
                    counter.pop(tree[left])
                left += 1       
            res = max(res,right-left+1)
        return res

## 48. Rotate Image
You are given an n x n 2D matrix representing an image.

Rotate the image by 90 degrees (clockwise).

Note:

You have to rotate the image in-place, which means you have to modify the input 2D matrix directly. DO NOT allocate another 2D matrix and do the rotation.

Example 1:

Given input matrix = 
[
  [1,2,3],
  [4,5,6],
  [7,8,9]
],

rotate the input matrix in-place such that it becomes:
[
  [7,4,1],
  [8,5,2],
  [9,6,3]
]

Example 2:

Given input matrix =
[
  [ 5, 1, 9,11],
  [ 2, 4, 8,10],
  [13, 3, 6, 7],
  [15,14,12,16]
], 

rotate the input matrix in-place such that it becomes:
[
  [15,13, 2, 5],
  [14, 3, 4, 1],
  [12, 6, 8, 9],
  [16, 7,10,11]
]

In [291]:
class Solution:
    def rotate(self, matrix) -> None:
        """
        Do not return anything, modify matrix in-place instead.
        """
        # O(N^2)
        for i in range(len(matrix)//2):
            for j in range(i,len(matrix)-1-i):
                top = matrix[i][j]
                matrix[i][j] = matrix[len(matrix)-1-j][i] # top -> left
                matrix[len(matrix)-1-j][i] = matrix[len(matrix)-1-i][len(matrix)-1-j] #left -> bottom
                matrix[len(matrix)-1-i][len(matrix)-1-j] = matrix[j][len(matrix)-1-i] #bottom -> right
                matrix[j][len(matrix)-1-i] = top #right -> top